In [ ]:
!pwd
!unzip 2021_GCP_all_for_NSW_short-header.zip -d census
!ls

Start lean to avoid exploding dimensions:

* Age × Sex (from G04)

* Income (from G17 or marginalised version)

* Occupation (from G60, possibly at major-group ANZSCO level)

* Language or Country of birth (from G09 or G13, grouped into Top N categories + “Other”)

* That gives a manageable 4D joint distribution (AGE × SEX × INCOME × OCCUPATION × CULTURAL) per SA2.


In [10]:
import pandas as pd
import glob
import re
import numpy as np
from itertools import product
import math
from typing import Dict, List, Optional, Tuple

In [6]:
tables = ['G04','G17','G60','G09']

for table in tables:
    # merge all parts of a table e.g. G04A and G04B
    files = sorted(glob.glob(f"/content/census/2021_Census_GCP_All_Geographies/SA2/NSW/2021Census_{table}*_NSW_SA2.csv"))
    print(files)
    dfs = []
    for f in files:
        print(f)
        df = pd.read_csv(f)
        #print(df.head())
        # Keep SA2_CODE_2021 plus all other columns except duplicated ones
        if 'SA2_CODE_2021' in df.columns:
            dfs.append(df)

    # Drop duplicate SA2_CODE_2021 columns automatically
    merged = dfs[0]
    for df in dfs[1:]:
        merged = merged.merge(df, on="SA2_CODE_2021")
    print(merged.columns)
    print(merged.shape)
    print(merged.head())
    merged.to_csv(f"/content/census/2021_Census_GCP_All_Geographies/SA2/NSW/2021Census_{table}_NSW_SA2_full.csv", index=False)

['/content/census/2021_Census_GCP_All_Geographies/SA2/NSW/2021Census_G04A_NSW_SA2.csv', '/content/census/2021_Census_GCP_All_Geographies/SA2/NSW/2021Census_G04B_NSW_SA2.csv']
/content/census/2021_Census_GCP_All_Geographies/SA2/NSW/2021Census_G04A_NSW_SA2.csv
/content/census/2021_Census_GCP_All_Geographies/SA2/NSW/2021Census_G04B_NSW_SA2.csv
Index(['SA2_CODE_2021', 'Age_yr_0_M', 'Age_yr_0_F', 'Age_yr_0_P', 'Age_yr_1_M',
       'Age_yr_1_F', 'Age_yr_1_P', 'Age_yr_2_M', 'Age_yr_2_F', 'Age_yr_2_P',
       ...
       'Age_yr_90_94_P', 'Age_yr_95_99_M', 'Age_yr_95_99_F', 'Age_yr_95_99_P',
       'Age_yr_100_yr_over_M', 'Age_yr_100_yr_over_F', 'Age_yr_100_yr_over_P',
       'Tot_M', 'Tot_F', 'Tot_P'],
      dtype='object', length=307)
(644, 307)
   SA2_CODE_2021  Age_yr_0_M  Age_yr_0_F  Age_yr_0_P  Age_yr_1_M  Age_yr_1_F  \
0      101021007          26          18          40          17          12   
1      101021008          56          46         102          62          50   
2      1010

To extract 4D joint distribution (AGE × SEX × INCOME × OCCUPATION × CULTURAL) per SA2

**What are required:**

age_sex (from G04): per-SA2 counts by age_band × sex (all ages).

age_sex_15p (optional): same but 15+ only (to align with income/occupation universes).

income (from G17): per-SA2 counts by income_band (universe = 15+).

occupation (from G60): per-SA2 counts by occupation major group (mostly employed 15+).

cob_grp (from G09): per-SA2 counts by country_of_birth (grouped) (all ages).

In [7]:
path = "/content/census/2021_Census_GCP_All_Geographies/SA2/NSW"
g04 = pd.read_csv(f"{path}/2021Census_G04_NSW_SA2_full.csv")  # Age by Sex (all ages)
g17 = pd.read_csv(f"{path}/2021Census_G17_NSW_SA2_full.csv")  # Income by Age by Sex (15+)
g60 = pd.read_csv(f"{path}/2021Census_G60_NSW_SA2_full.csv")  # Occupation by Age by Sex (15+ employed)
g09 = pd.read_csv(f"{path}/2021Census_G09_NSW_SA2_full.csv")  # Country of Birth by Age by Sex (all ages)

key = "SA2_CODE_2021"

# ---------- G04: build Age × Sex (all ages) ----------
age_male_cols   = [c for c in g04.columns if re.match(r"^Age_yr_.*_M$", c)]
age_female_cols = [c for c in g04.columns if re.match(r"^Age_yr_.*_F$", c)]
age_person_cols = [c for c in g04.columns if re.match(r"^Age_yr_.*_P$", c)]

# Tidy to long (Age × Sex)
def tidy_g04_age_sex(df):
    recs = []
    for c in age_male_cols + age_female_cols:
        sex = 'Male' if c.endswith('_M') else 'Female'
        age = c.replace('Age_yr_', '').replace('_M','').replace('_F','')
        recs.append(
            df[[key, c]].assign(age_band=age, sex=sex).rename(columns={c:'count'})
        )
    out = pd.concat(recs, ignore_index=True)
    return out[[key, 'age_band', 'sex', 'count']]

age_sex = tidy_g04_age_sex(g04)  # constraint: AGE × SEX (all ages)

# Optional Age marginal (Persons)
age_person = (
    g04[[key] + age_person_cols]
    .melt(id_vars=[key], var_name='col', value_name='count')
    .assign(age_band=lambda d: d['col'].str.replace(r'^Age_yr_|_P$', '', regex=True))
    [[key,'age_band','count']]
)

# ---------- G17: Income marginal (Persons, 15+) ----------
# Persons totals per band columns end with _Tot and start with P_,
# but exclude P_Tot_* which are overall totals by age buckets, not income bands.
inc_band_cols = [c for c in g17.columns
                 if c.startswith('P_') and c.endswith('_Tot') and not c.startswith('P_Tot_')]

income = (
    g17[[key] + inc_band_cols]
    .melt(id_vars=[key], var_name='income_band', value_name='count')
    [[key,'income_band','count']]
)

# ---------- (Optional) align bases: use 15+ slice of G04 for AGE × SEX 15+ ----------
# might add a joint constraint with income, filter G04 to 15+ ages.
def is_15plus(age_label):
    # G04 uses single years and grouped (e.g., '0', '1', ... '90_94', '100_yr_over')
    # Keep labels that represent ages >= 15 (single) or groups whose lower bound >= 15.
    if age_label.isdigit():
        return int(age_label) >= 15
    if '_' in age_label:
        low = int(re.split(r'_|yr', age_label)[0])  # crude parse
        return low >= 15
    if 'over' in age_label:
        return True
    return False

age_sex_15p = age_sex[age_sex['age_band'].apply(is_15plus)]


# ---------- G60: Occupation marginal (Persons) ----------
occ_person_tot_cols = [c for c in g60.columns if c.startswith('P_Tot_') and not c.endswith('_Tot')]  # e.g., P_Tot_Managers
# Some tables also include P_Tot_Tot (grand total) — drop it if present
occ_person_tot_cols = [c for c in occ_person_tot_cols if c != 'P_Tot_Tot']

occupation = (
    g60[[key] + occ_person_tot_cols]
    .melt(id_vars=[key], var_name='occ_raw', value_name='count')
    .assign(occupation=lambda d: d['occ_raw'].str.replace(r'^P_Tot_', '', regex=True))
    [[key,'occupation','count']]
)

# ---------- G09: Country of Birth (Persons Totals) ----------
# Keep all *_Tot under P_XXXX_Tot
cob_tot_cols = [c for c in g09.columns if c.startswith('P_') and c.endswith('_Tot')]

cob = (
    g09[[key] + cob_tot_cols]
    .melt(id_vars=[key], var_name='cob_col', value_name='count')
    .assign(country=lambda d: d['cob_col'].str.replace(r'^P_|_Tot$', '', regex=True))
    [[key,'country','count']]
)


Replace SA_CODE with name i.e. Census_Code_2021 --> Census_Name_2021

In [ ]:
# ---------- Read the mapping from the Excel ----------
geo_xlsx = "/content/census/Metadata/2021Census_geog_desc_1st_2nd_3rd_release.xlsx"
sheet = "2021_ASGS_MAIN_Structures"

# Read only the needed columns + filter to SA2 rows
geo = pd.read_excel(
    geo_xlsx, sheet_name=sheet, dtype=str, engine="openpyxl"
)[["ASGS_Structure","Census_Code_2021","AGSS_Code_2021","Census_Name_2021"]]
geo = geo[geo["ASGS_Structure"] == "SA2"].copy()

# Normalise codes to 9-digit strings
geo["Census_Code_2021"] = geo["Census_Code_2021"].str.zfill(9)
geo["AGSS_Code_2021"]   = geo["AGSS_Code_2021"].str.zfill(9)

# ---------- Check if the 2nd and 3rd columns are identical ----------
codes_equal = (geo["Census_Code_2021"] == geo["AGSS_Code_2021"]).all()
print("Census_Code_2021 == AGSS_Code_2021 for all SA2 rows:", codes_equal)

# If not equal (unexpected), you can inspect mismatches:
if not codes_equal:
    mismatches = geo.loc[geo["Census_Code_2021"] != geo["AGSS_Code_2021"]]
    print("Mismatched rows:\n", mismatches.head())

# Build mapping dict: SA2_CODE_2021 (string) -> SA2 name
sa2_to_name = dict(zip(geo["Census_Code_2021"], geo["Census_Name_2021"]))

# ---------- Apply to extracted tables ----------
# ensure SA2_CODE_2021 is string in CSV loads
# def ensure_code_str(df, col="SA2_CODE_2021"):
#     df[col] = df[col].astype(str).str.zfill(9)
#     return df

# path = "/content/census/2021_Census_GCP_All_Geographies/SA2/NSW"
# g04 = pd.read_csv(f"{path}/2021Census_G04_NSW_SA2_full.csv")  # Age by Sex (all ages)
# g17 = pd.read_csv(f"{path}/2021Census_G17_NSW_SA2_full.csv")  # Income by Age by Sex (15+)
# g60 = pd.read_csv(f"{path}/2021Census_G60_NSW_SA2_full.csv")  # Occupation by Age by Sex (15+ employed)
# g09 = pd.read_csv(f"{path}/2021Census_G09_NSW_SA2_full.csv")  # Country of Birth by Age by Sex (all ages)

# g04 = ensure_code_str(g04, "SA2_CODE_2021")
# g17 = ensure_code_str(g17, "SA2_CODE_2021")
# g60 = ensure_code_str(g60, "SA2_CODE_2021")
# g09 = ensure_code_str(g09, "SA2_CODE_2021")

for df in [age_sex, age_sex_15p, age_person, income, occupation, cob, cob_grp]:
    if df is not None and "SA2_CODE_2021" in df.columns:
        df["SA2_NAME_2021"] = df["SA2_CODE_2021"].map(sa2_to_name)


In [8]:
# Example: collapse to Top-K countries at NSW level (+ Elsewhere + Not stated)
topK = 15
cob_state_totals = cob.groupby('country')['count'].sum().sort_values(ascending=False)
keep = set(cob_state_totals.head(topK).index) | {'Elsewhere','COB_NS'}
cob['country_grouped'] = cob['country'].where(cob['country'].isin(keep), other='Elsewhere')
cob_grp = cob.groupby([key,'country_grouped'], as_index=False)['count'].sum()
cob_grp = cob_grp.rename(columns={'country_grouped':'country'})

In [9]:
cob_grp

,SA2_CODE_2021,country,count
0,101021007,Australia,3422
1,101021007,COB_NS,387
2,101021007,China,4
3,101021007,Elsewhere,238
4,101021007,England,177
...,...,...,...
9655,199999499,New_Zealand,194
9656,199999499,Philippines,50
9657,199999499,South_Africa,55
9658,199999499,Tot,11371


**Population Model**

1. Define categories
2. Create constraints (targets)
3. Build and seed a 4D joint - `X ∝ AgeSex[a,s] × Income[i] × Occupation[o]`
4. Run IPF
5. Integerize
6. Generate individuals
    * person_id | SA2 | age_band | sex | income_band | occupation
7. Validate

In [12]:
# -----------------------------
# Step 1: Universe + age bands
# -----------------------------

AGE_BANDS_15P = [
    ("15_19", 15, 19),
    ("20_24", 20, 24),
    ("25_34", 25, 34),
    ("35_44", 35, 44),
    ("45_54", 45, 54),
    ("55_64", 55, 64),
    ("65_74", 65, 74),
    ("75_84", 75, 84),
    ("85ov", 85, 200),
]

def map_age_label_to_band(age_label: str) -> Optional[str]:
    """
    Map G04-style labels (e.g., '15', '50_54', '100_yr_over') to 15+ bands.
    Returns None for <15.
    """
    s = str(age_label)
    if s.isdigit():
        x = int(s)
        for lab, lo, hi in AGE_BANDS_15P:
            if lo <= x <= hi:
                return lab
        return None
    if "_" in s:
        parts = s.split("_")
        try:
            lo = int(parts[0])
        except ValueError:
            lo = None
        if lo is not None:
            for lab, lo2, hi2 in AGE_BANDS_15P:
                if lo2 <= lo <= hi2:
                    return lab
    if "over" in s or "ov" in s:
        return "85ov"
    return None

# -------------------------------------
# Step 2: Build constraints per SA2
# -------------------------------------

def build_age_sex_constraint(sa2: str,
                             age_sex_long: pd.DataFrame,
                             fifteen_plus: bool=True,
                             band_to_15p: bool=True) -> pd.DataFrame:
    """
    Returns A: DataFrame index=age_band, columns=['Male','Female'], values=count.
    age_sex_long columns: ['SA2_CODE_2021','age_band','sex','count']
    """
    df = age_sex_long[age_sex_long["SA2_CODE_2021"] == sa2].copy()
    if fifteen_plus:
        df["age_band_15p"] = df["age_band"].map(map_age_label_to_band)
        df = df.dropna(subset=["age_band_15p"])
        if band_to_15p:
            df = (df.groupby(["age_band_15p","sex"], as_index=False)["count"].sum()
                    .rename(columns={"age_band_15p":"age_band"}))
    A = df.pivot(index="age_band", columns="sex", values="count").fillna(0)
    # Ensure both columns exist
    for s in ["Male","Female"]:
        if s not in A.columns:
            A[s] = 0
    return A[["Male","Female"]]

def build_income_constraint(sa2: str, income_long: pd.DataFrame) -> pd.Series:
    """
    Returns I: Series index=income_band, values=count (15+).
    """
    I = (income_long[income_long["SA2_CODE_2021"] == sa2]
         .set_index("income_band")["count"]
         .astype(float).fillna(0)).sort_index()
    return I

def build_occupation_constraint(sa2: str, occupation_long: pd.DataFrame) -> pd.Series:
    """
    Returns O: Series index=occupation (ANZSCO majors), values=count.
    """
    O = (occupation_long[occupation_long["SA2_CODE_2021"] == sa2]
         .set_index("occupation")["count"]
         .astype(float).fillna(0)).sort_index()
    return O

def build_cob_constraint(sa2: str, cob_long: pd.DataFrame) -> pd.Series:
    """
    Returns C: Series index=country (grouped), values=count.
    """
    C = (cob_long[cob_long["SA2_CODE_2021"] == sa2]
         .set_index("country")["count"]
         .astype(float).fillna(0)).sort_index()
    return C

# -------------------------------------
# Step 3–4: Seed the joint tensor X
# -------------------------------------

def seed_joint(A: pd.DataFrame,
               I: Optional[pd.Series]=None,
               O: Optional[pd.Series]=None,
               C: Optional[pd.Series]=None) -> Tuple[np.ndarray, List[str], List[str], List[str], List[str], List[str]]:
    """
    Build initial independent joint X with dims among:
    A (age×sex), I (income), O (occupation), C (country).
    Returns X and label lists (ages, sexes, incs, occs, cobs).
    Order of axes: (a, s, i?, o?, c?)
    """
    ages  = list(A.index)
    sexes = list(A.columns)
    A_np  = A.values.astype(float)

    incs, occs, cobs = [], [], []
    X = A_np.copy()

    if I is not None:
        incs = list(I.index)
        I_np = I.values.astype(float)
        X = X[:, :, None] * I_np[None, None, :]

    if O is not None:
        occs = list(O.index)
        O_np = O.values.astype(float)
        if X.ndim == 2:
            X = X[:, :, None]
        X = X[:, :, :, None] * O_np[None, None, None, :]

    if C is not None:
        cobs = list(C.index)
        C_np = C.values.astype(float)
        if X.ndim == 2:
            X = X[:, :, None]
        if X.ndim == 3:
            X = X[:, :, :, None]
        if X.ndim == 4:
            X = X[:, :, :, :, None]
        X = X * C_np[None, None, None, None, :]

    # Normalise to Age×Sex total
    X = X * (A_np.sum() / (X.sum() + 1e-12))
    return X, ages, sexes, incs, occs, cobs

# -----------------------------
# Step 5: IPF (raking)
# -----------------------------

def ipf_match(X: np.ndarray,
              A: Optional[np.ndarray]=None,   # Age×Sex target
              I: Optional[np.ndarray]=None,   # Income target
              O: Optional[np.ndarray]=None,   # Occupation target
              C: Optional[np.ndarray]=None,   # Country target
              tol: float=1e-3,
              max_iter: int=60,
              ridge: float=0.0) -> np.ndarray:
    """
    Rake X so marginals match provided targets.
    Axis order assumed: (a, s, i?, o?, c?)
    """
    eps = 1e-12
    for _ in range(max_iter):
        errs = []

        # Age×Sex (sum over trailing axes)
        if A is not None:
            cur = X.sum(axis=tuple(range(2, X.ndim)))
            alpha = (A + ridge) / (cur + ridge + eps)
            X = X * alpha[(slice(None), slice(None)) + (None,)*(X.ndim-2)]
            errs.append(np.max(np.abs(cur - A) / (A + eps)))

        # Income (sum over a,s and any axes after i)
        if I is not None and X.ndim >= 3:
            cur = X.sum(axis=(0,1) + tuple(range(3, X.ndim)))
            beta = (I + ridge) / (cur + ridge + eps)
            X = X * beta[(None,None,slice(None)) + (None,)*(X.ndim-3)]
            errs.append(np.max(np.abs(cur - I) / (I + eps)))

        # Occupation (sum over a,s,i and any after o)
        if O is not None and X.ndim >= 4:
            cur = X.sum(axis=(0,1,2) + tuple(range(4, X.ndim)))
            gamma = (O + ridge) / (cur + ridge + eps)
            X = X * gamma[(None,None,None,slice(None)) + (None,)*(X.ndim-4)]
            errs.append(np.max(np.abs(cur - O) / (O + eps)))

        # Country of Birth (sum over a,s,i,o)
        if C is not None and X.ndim >= 5:
            cur = X.sum(axis=(0,1,2,3))
            delta = (C + ridge) / (cur + ridge + eps)
            X = X * delta[(None,None,None,None,slice(None))]
            errs.append(np.max(np.abs(cur - C) / (C + eps)))

        if errs and max(errs) < tol:
            break
    return X

# -----------------------------------
# Step 6: Integerise (largest remainder)
# -----------------------------------

def integerise_largest_remainder(X: np.ndarray) -> np.ndarray:
    floored = np.floor(X).astype(int)
    rem = X - floored
    to_assign = int(round(X.sum() - floored.sum()))
    if to_assign <= 0:
        return floored
    idx = np.argsort(rem.ravel())[::-1][:to_assign]
    out = floored.ravel()
    out[idx] += 1
    return out.reshape(X.shape)

# -----------------------------
# Step 7: Generate individuals
# -----------------------------

def generate_individual_rows(X_int: np.ndarray,
                             ages: List[str], sexes: List[str],
                             incs: List[str], occs: List[str], cobs: List[str],
                             sa2_name: str) -> pd.DataFrame:
    recs = []
    if X_int.ndim == 2:  # A×S
        for ia,a in enumerate(ages):
            for isx,s in enumerate(sexes):
                recs += [[sa2_name,a,s,None,None,None]] * int(X_int[ia,isx])
    elif X_int.ndim == 3:  # A×S×I
        for ia,a in enumerate(ages):
            for isx,s in enumerate(sexes):
                for ii,i in enumerate(incs):
                    recs += [[sa2_name,a,s,i,None,None]] * int(X_int[ia,isx,ii])
    elif X_int.ndim == 4:  # A×S×I×O
        for ia,a in enumerate(ages):
            for isx,s in enumerate(sexes):
                for ii,i in enumerate(incs):
                    for io,o in enumerate(occs):
                        recs += [[sa2_name,a,s,i,o,None]] * int(X_int[ia,isx,ii,io])
    elif X_int.ndim == 5:  # A×S×I×O×C
        for ia,a in enumerate(ages):
            for isx,s in enumerate(sexes):
                for ii,i in enumerate(incs):
                    for io,o in enumerate(occs):
                        for ic,c in enumerate(cobs):
                            recs += [[sa2_name,a,s,i,o,c]] * int(X_int[ia,isx,ii,io,ic])

    people = pd.DataFrame(recs, columns=["SA2_NAME_2021","age_band","sex","income_band","occupation","country"])
    people.insert(0, "person_id", [f"P{str(i).zfill(8)}" for i in range(len(people))])
    return people

# -----------------------------
# Step 8: Validation
# -----------------------------

def validate_against_constraints(df_people: pd.DataFrame,
                                 A: Optional[pd.DataFrame]=None,
                                 I: Optional[pd.Series]=None,
                                 O: Optional[pd.Series]=None,
                                 C: Optional[pd.Series]=None) -> pd.DataFrame:
    """
    Returns tidy report with abs/rel errors for each enforced marginal.
    """
    reports = []
    if A is not None:
        curA = (df_people.groupby(["age_band","sex"]).size().unstack(fill_value=0))
        curA = curA.reindex(index=A.index, columns=A.columns, fill_value=0)
        diff = (curA - A).stack().rename("abs_err")
        rel  = (diff.abs() / (A.stack().replace(0, np.nan))).rename("rel_err")
        r = pd.concat([diff, rel], axis=1).reset_index().rename(columns={"level_0":"age_band","level_1":"sex"})
        r["constraint"] = "Age×Sex"
        reports.append(r)

    if I is not None:
        curI = df_people.groupby("income_band").size()
        curI = curI.reindex(I.index, fill_value=0)
        r = pd.DataFrame({"income_band": I.index,
                          "abs_err": (curI - I).values,
                          "rel_err": np.where(I.values==0, 0, np.abs(curI - I)/I.values)})
        r["constraint"] = "Income"
        reports.append(r)

    if O is not None:
        curO = df_people.groupby("occupation").size()
        curO = curO.reindex(O.index, fill_value=0)
        r = pd.DataFrame({"occupation": O.index,
                          "abs_err": (curO - O).values,
                          "rel_err": np.where(O.values==0, 0, np.abs(curO - O)/O.values)})
        r["constraint"] = "Occupation"
        reports.append(r)

    if C is not None:
        curC = df_people.groupby("country").size()
        curC = curC.reindex(C.index, fill_value=0)
        r = pd.DataFrame({"country": C.index,
                          "abs_err": (curC - C).values,
                          "rel_err": np.where(C.values==0, 0, np.abs(curC - C)/C.values)})
        r["constraint"] = "Country"
        reports.append(r)

    return pd.concat(reports, ignore_index=True) if reports else pd.DataFrame()

# ----------------------------------------------------
# Orchestrator: build population for one SA2
# ----------------------------------------------------

def build_population_for_sa2(sa2_code: str,
                             sa2_name: str,
                             universe: str,                       # 'all' or '15plus'
                             age_sex_long: pd.DataFrame,
                             income_long: Optional[pd.DataFrame]=None,
                             occupation_long: Optional[pd.DataFrame]=None,
                             cob_long: Optional[pd.DataFrame]=None,
                             include_occupation: bool=False,
                             include_country: bool=False,
                             tol: float=1e-3,
                             ridge: float=0.0) -> Tuple[pd.DataFrame, pd.DataFrame]:
    """
    Builds the synthetic population for one SA2 and returns:
      (people_df, validation_report_df)
    """
    if universe not in ("all","15plus"):
        raise ValueError("universe must be 'all' or '15plus'")

    # Constraints
    A = build_age_sex_constraint(sa2_code, age_sex_long,
                                 fifteen_plus=(universe=="15plus"),
                                 band_to_15p=(universe=="15plus"))
    I = build_income_constraint(sa2_code, income_long) if (universe=="15plus" and income_long is not None) else None
    O = build_occupation_constraint(sa2_code, occupation_long) if (include_occupation and occupation_long is not None) else None
    C = build_cob_constraint(sa2_code, cob_long) if (include_country and cob_long is not None) else None

    # Seed & IPF
    X0, ages, sexes, incs, occs, cobs = seed_joint(A, I, O, C)
    Xf = ipf_match(X0.copy(),
                   A=A.values.astype(float),
                   I=None if I is None else I.values.astype(float),
                   O=None if O is None else O.values.astype(float),
                   C=None if C is None else C.values.astype(float),
                   tol=tol, max_iter=60, ridge=ridge)

    # Integerise & Individuals
    X_int = integerise_largest_remainder(Xf)
    people = generate_individual_rows(X_int, ages, sexes, incs, occs, cobs, sa2_name)

    # Validation
    report = validate_against_constraints(people, A=A, I=I, O=O, C=C)
    return people, report

In [15]:
# EXAMPLE USAGE
people, report = build_population_for_sa2(
    sa2_code="101021008",
    sa2_name="Karabar",
    universe="15plus",              # using G17 + G60
    age_sex_long=age_sex,           # G04 tidy
    income_long=income,             # G17 tidy
    occupation_long=occupation,     # G60 tidy
    cob_long=None,                  # skip G09 for now (different universe)
    include_occupation=True,
    include_country=False,
    tol=1e-3,
    ridge=0.2
)

ValueError: zero-size array to reduction operation maximum which has no identity

In [14]:
# Which SA2s exist in the tables you actually loaded?
sa2s_age = age_sex['SA2_CODE_2021'].unique()
sa2s_inc = income['SA2_CODE_2021'].unique()
sa2s_occ = occupation['SA2_CODE_2021'].unique()

print("Counts:", len(sa2s_age), len(sa2s_inc), len(sa2s_occ))
print("Has 213031350 in age_sex?", "213031350" in set(sa2s_age))

# Show a few valid NSW SA2 codes you can use:
print(sorted(sa2s_age[:10]))

Counts: 644 644 644
Has 213031350 in age_sex? False
[np.int64(101021007), np.int64(101021008), np.int64(101021009), np.int64(101021010), np.int64(101021012), np.int64(101021610), np.int64(101021611), np.int64(101031013), np.int64(101031014), np.int64(101031015)]


In [ ]:
# Collapse age_sex_15p single-year ages into 5-year bands (15–19, 20–24, …) by summing
# Band map (adjust to what G17/G60 use)
bins = [(15,19),(20,24),(25,34),(35,44),(45,54),(55,64),(65,74),(75,84),(85,200)]
labels = ["15_19","20_24","25_34","35_44","45_54","55_64","65_74","75_84","85ov"]

def age_to_band(a):
    # a is like '15', '16', '90_94', '100_yr_over'
    s = str(a)
    if s.isdigit():
        x = int(s)
        for (lo,hi),lab in zip(bins,labels):
            if lo <= x <= hi:
                return lab
    if '_' in s and s.replace('_','').isdigit():
        lo = int(s.split('_')[0])
        for (lo2,hi2),lab in zip(bins,labels):
            if lo2 <= lo <= hi2:
                return lab
    return "85ov"  # fallback for 90_94 / 100_yr_over

age_sex_banded = (
    age_sex_15p.assign(age_band=lambda d: d['age_band'].map(age_to_band))
               .groupby(['SA2_CODE_2021','age_band','sex'], as_index=False)['count'].sum()
)

NameError: name 'age_sex_banded' is not defined